#  프롬프트 엔지니어링 
- Chain-of-Thought (CoT) 등 고급 기법 

---

# 환경 설정 및 준비

`(1) Env 환경변수`

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

`(2) 기본 라이브러리`

In [2]:
import os
from glob import glob

from pprint import pprint
import json

`(3) LLM 설정`

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model='gpt-4.1-nano',
    temperature=0.3,
    top_p=0.9,
)

/Users/kwonsoonhyun/Sesac/004_llm_agent/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.4.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/kwonsoonhyun/Sesac/004_llm_agent/.venv/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_

In [ ]:
from langchain_ollama import ChatOllama

ollama = ChatOllama(
    model='qwen2.5:3b',
    temperature=0.3,
    top_p=0.9,
)

#모델파인튜닝할때 cot를 많이 바꾼다.

# **Chain of Thought (CoT)**

* Chain of Thought는 AI 모델이 복잡한 문제를 해결할 때 각 단계별 사고 과정을 명시적으로 보여주도록 하는 프롬프팅 기법으로, 이를 통해 모델의 추론 과정을 투명하게 확인할 수 있고 더 정확한 결과를 도출할 수 있습니다.

* 이 방식은 특히 수학 문제 풀이, 논리적 추론이 필요한 과제, 복잡한 의사결정 과정에서 매우 효과적이며, 모델이 중간 단계에서 발생할 수 있는 오류를 스스로 발견하고 수정할 수 있게 합니다.

* CoT의 주요 장점은 문제 해결 과정의 투명성을 높이고 최종 답변의 신뢰성을 향상시킬 수 있다는 것이지만, 출력이 길어지고 계산 비용이 증가할 수 있다는 단점도 존재합니다.


`(1) Zero-shot 프롬프팅`

   - 가장 단순한 형태의 프롬프팅
   - 예시나 단계별 설명 없이 직접 답을 출력
   - 속도가 빠르고 메모리 사용량이 적은 편
   - 단순한 문제에 적합

In [6]:
from langchain_core.prompts import PromptTemplate

# 프롬프트 템플릿 생성
zero_shot_template = """
다음 문제를 해결하시오:

문제: {question}

답안:
"""

zero_shot_prompt = PromptTemplate(
    input_variables=["question"],
    template=zero_shot_template
)

# 테스트용 문제
question = """
학교에서 500명의 학생이 있습니다. 이 중 30%는 5학년이고, 20%는 6학년 학생입니다. 
5학년 학생들 중 60%는 수학 동아리에 있고, 나머지는 과학 동아리에 있습니다. 
6학년 학생들 중 70%는 수학 동아리에 있고, 나머지는 과학 동아리에 있습니다.
과학 동아리에는 몇 명의 학생이 있나요?
"""

# 프롬프트 출력
print(zero_shot_prompt.format(question=question))


다음 문제를 해결하시오:

문제: 
학교에서 500명의 학생이 있습니다. 이 중 30%는 5학년이고, 20%는 6학년 학생입니다. 
5학년 학생들 중 60%는 수학 동아리에 있고, 나머지는 과학 동아리에 있습니다. 
6학년 학생들 중 70%는 수학 동아리에 있고, 나머지는 과학 동아리에 있습니다.
과학 동아리에는 몇 명의 학생이 있나요?


답안:



In [7]:
# OpenAI GPT-4.1-nano 모델로 답안 생성

zero_shot_chain = zero_shot_prompt | llm 

answer = zero_shot_chain.invoke({"question": question})

print(answer.content)

먼저 전체 학생 수는 500명입니다.

1. 5학년 학생 수:
   30% → 0.30 × 500 = 150명

2. 6학년 학생 수:
   20% → 0.20 × 500 = 100명

3. 5학년 학생 중 수학 동아리 학생 수:
   60% → 0.60 × 150 = 90명

4. 5학년 학생 중 과학 동아리 학생 수:
   나머지 → 150 - 90 = 60명

5. 6학년 학생 중 수학 동아리 학생 수:
   70% → 0.70 × 100 = 70명

6. 6학년 학생 중 과학 동아리 학생 수:
   나머지 → 100 - 70 = 30명

이제 과학 동아리 학생 수는 5학년 과학 + 6학년 과학입니다:
60 + 30 = 90명

**답: 과학 동아리에는 90명의 학생이 있습니다.**


In [8]:
# Ollama qwen2.5:3b 모델로 답안 생성

zero_shot_chain = zero_shot_prompt | ollama

answer = zero_shot_chain.invoke({"question": question})

print(answer.content)

ConnectError: [Errno 61] Connection refused

`(2) One-shot/Few-shot 프롬프팅`

   - 하나 이상의 예시를 통해 문제 해결 방식을 제시 
   - 유사한 예시를 통해 학습 효과를 기대
   - Zero-shot보다 더 정확한 결과를 얻을 수 있음
   - 중간 복잡도의 문제에 적합

   - 논문: https://arxiv.org/abs/2005.14165

In [ ]:
from langchain_core.prompts import PromptTemplate

# 프롬프트 템플릿 생성
one_shot_template = """
다음은 수학 문제를 해결하는 예시입니다:

예시 문제: 한 학급에 30명의 학생이 있습니다. 이 중 40%가 남학생이라면, 여학생은 몇 명인가요?

예시 풀이:
1) 먼저 남학생 수를 계산합니다:
   - 전체 학생의 40% = 30 x 0.4 = 12명이 남학생

2) 여학생 수를 계산합니다:
   - 전체 학생 수 - 남학생 수 = 30 - 12 = 18명이 여학생

따라서 여학생은 18명입니다.

이제 아래 문제를 같은 방식으로 해결하시오:

새로운 문제: {question}

답안:
"""

one_shot_prompt = PromptTemplate(
   input_variables=["question"],
   template=one_shot_template
)

# 테스트용 문제
question = """
학교에서 500명의 학생이 있습니다. 이 중 30%는 5학년이고, 20%는 6학년 학생입니다. 
5학년 학생들 중 60%는 수학 동아리에 있고, 나머지는 과학 동아리에 있습니다. 
6학년 학생들 중 70%는 수학 동아리에 있고, 나머지는 과학 동아리에 있습니다.
과학 동아리에는 몇 명의 학생이 있나요?
"""

# 프롬프트 출력
print(one_shot_prompt.format(question=question))

In [ ]:
# OpenAI GPT-4.1-nano 모델로 답안 생성

one_shot_chain = one_shot_prompt | llm 

answer = one_shot_chain.invoke({"question": question})

print(answer.content)

In [ ]:
# Ollama qwen2.5:3b 모델로 답안 생성

one_shot_chain = one_shot_prompt | ollama

answer = one_shot_chain.invoke({"question": question})

print(answer.content)

`(3) Chain of Thought(CoT) 프롬프팅`

   - 가장 체계적인 문제 해결 방식을 제공
   - 명시적인 단계별 추론 과정을 포함
   - 복잡한 문제 해결에 가장 적합

   - 논문: https://arxiv.org/abs/2201.11903

In [ ]:
from langchain_core.prompts import PromptTemplate

# 프롬프트 템플릿 생성
cot_template = """
수학 문제를 해결하기 위해 다음 단계를 따라하세요:
문제: {question}

해결 전략: 백분율 계산 문제는 전체에서 부분을 구하는 문제입니다.

📝 1단계: 주어진 정보를 표로 정리하기
전체 학생 수: 
5학년 비율: 
6학년 비율: 
5학년 중 수학 동아리 비율:
5학년 중 과학 동아리 비율: (100% - 수학 동아리 비율)
6학년 중 수학 동아리 비율:
6학년 중 과학 동아리 비율: (100% - 수학 동아리 비율)

🔢 2단계: 백분율을 소수로 변환하기
- 30% = 30/100 = 0.3
- 20% = 20/100 = 0.2
- 60% = 60/100 = 0.6
- 70% = 70/100 = 0.7

📊 3단계: 각 학년별 학생 수 계산하기
5학년 학생 수 = 500 × 0.3 = ?
6학년 학생 수 = 500 × 0.2 = ?

🎯 4단계: 과학 동아리 비율 계산하기
5학년 과학 동아리 비율 = 100% - 60% = 40% = 0.4
6학년 과학 동아리 비율 = 100% - 70% = 30% = 0.3

🧮 5단계: 각 학년의 과학 동아리 학생 수 계산하기
5학년 과학 동아리 학생 수 = (5학년 학생 수) × 0.4
6학년 과학 동아리 학생 수 = (6학년 학생 수) × 0.3

➕ 6단계: 전체 과학 동아리 학생 수 구하기
전체 과학 동아리 학생 수 = 5학년 과학 동아리 + 6학년 과학 동아리

✅ 7단계: 답안 검증하기
- 계산 실수는 없는가?
- 답이 합리적인가?

최종 답안: 과학 동아리에는 ___명의 학생이 있습니다.
"""

cot_prompt = PromptTemplate(
    input_variables=["question"],
    template=cot_template
)

# 테스트용 문제
question = """
학교에서 500명의 학생이 있습니다. 이 중 30%는 5학년이고, 20%는 6학년 학생입니다. 
5학년 학생들 중 60%는 수학 동아리에 있고, 나머지는 과학 동아리에 있습니다. 
6학년 학생들 중 70%는 수학 동아리에 있고, 나머지는 과학 동아리에 있습니다.
과학 동아리에는 몇 명의 학생이 있나요?
"""

# 프롬프트 출력
print(cot_prompt.format(question=question))

In [ ]:
# OpenAI GPT-4.1-nano 모델로 답안 생성

cot_chain = cot_prompt | llm 

answer = cot_chain.invoke({"question": question})

print(answer.content)

In [ ]:
# Ollama qwen2.5:3b 모델로 답안 생성

cot_chain = cot_prompt | ollama

answer = cot_chain.invoke({"question": question})

print(answer.content)

# **Self-Consistency**

* Self-Consistency는 AI 모델에게 하나의 문제에 대해 다양한 접근 방식으로 해결하도록 요청하는 기법으로, 여러 경로를 통해 도출된 결과들의 일관성을 확인함으로써 답변의 신뢰성을 높입니다.

* 이 방법은 특히 수학 문제나 논리적 추론이 필요한 과제에서 효과적이며, 서로 다른 방법으로 도출된 결과가 일치하는지 검증함으로써 오류 가능성을 최소화할 수 있습니다.

* Self-Consistency의 장점은 답변의 정확성을 높일 수 있다는 것이지만, 여러 번의 계산과 추론이 필요하므로 처리 시간이 길어지고 컴퓨팅 리소스 사용량이 증가한다는 단점이 있습니다.

* 또한 이 기법은 Chain of Thought (CoT) 프롬프팅과 결합하여 사용할 경우 더욱 강력한 효과를 발휘할 수 있습니다.

- 논문: https://arxiv.org/abs/2203.11171


In [ ]:
from langchain_core.prompts import PromptTemplate

# 프롬프트 템플릿 생성
self_consistency_template = """
다음 문제를 세 가지 다른 방법으로 해결하시오:

문제: {question}

세 가지 풀이 방법:
1) 직접 계산 방법:
   - 주어진 숫자를 직접 계산

2) 비율 활용 방법:
   - 전체에 대한 비율로 계산

3) 단계별 분해 방법:
   - 문제를 작은 부분으로 나누어 계산

각 방법의 답안을 제시하고, 결과가 일치하는지 확인하시오.

답안:
"""

self_consistency_prompt = PromptTemplate(
   input_variables=["question"],
   template=self_consistency_template
)

# 테스트용 문제
question = """
학교에서 500명의 학생이 있습니다. 이 중 30%는 5학년이고, 20%는 6학년 학생입니다. 
5학년 학생들 중 60%는 수학 동아리에 있고, 나머지는 과학 동아리에 있습니다. 
6학년 학생들 중 70%는 수학 동아리에 있고, 나머지는 과학 동아리에 있습니다.
과학 동아리에는 몇 명의 학생이 있나요?
"""

# OpenAI gpt-4.1-nano 모델로 답안 생성
self_consistency_chain = self_consistency_prompt | llm 
answer = self_consistency_chain.invoke({"question": question})

print(answer.content)

In [ ]:
# Ollama qwen2.5:3b 모델로 해결
self_consistency_chain = self_consistency_prompt | ollama
answer = self_consistency_chain.invoke({"question": question})

print(answer.content)

# **Program-Aided Language (PAL)**

* PAL은 자연어 문제를 프로그래밍적 사고방식으로 접근하도록 하는 기법으로, 복잡한 문제를 코드나 의사코드 형태로 분해하여 해결하는 방식입니다. 이를 통해 문제 해결 과정을 더욱 구조화하고 체계적으로 만들 수 있습니다.

* 이 접근 방식의 큰 장점은 프로그래밍 언어의 정확성과 논리성을 활용하여 모호함을 줄이고, 각 단계를 명확하게 정의할 수 있다는 것입니다. 특히 수학적 계산, 데이터 처리, 알고리즘적 문제 해결에서 뛰어난 성능을 보입니다.

* PAL의 특징적인 점은 실제 실행 가능한 코드를 생성할 수 있다는 것으로, 이는 결과의 검증이 용이하고 필요한 경우 수정이나 최적화가 가능하다는 장점이 있습니다. 

- 논문: https://arxiv.org/pdf/2211.10435


In [ ]:
from langchain_core.prompts import PromptTemplate

# 프롬프트 템플릿 생성
pal_template = """
다음 문제를 Python 프로그래밍 방식으로 해결하시오:

문제: {question}

# 문제 해결을 위한 Python 스타일 의사코드:
def solve_problem():
    # 1. 변수 정의
    # - 주어진 값들을 변수로 저장
    
    # 2. 계산 과정
    # - 필요한 계산을 단계별로 수행
    
    # 3. 결과 반환
    # - 최종 결과 계산 및 반환
    
답안:
"""

pal_prompt = PromptTemplate(
    input_variables=["question"],
    template=pal_template
)

# 테스트용 문제
question = """
학교에서 500명의 학생이 있습니다. 이 중 30%는 5학년이고, 20%는 6학년 학생입니다. 
5학년 학생들 중 60%는 수학 동아리에 있고, 나머지는 과학 동아리에 있습니다. 
6학년 학생들 중 70%는 수학 동아리에 있고, 나머지는 과학 동아리에 있습니다.
과학 동아리에는 몇 명의 학생이 있나요?
"""

# OpenAI GPT-4.1-nano 모델로 답안 생성
pal_chain = pal_prompt | llm 
answer = pal_chain.invoke({"question": question})

print(answer.content)

In [ ]:
def solve_problem():
    # 1. 변수 정의
    total_students = 500
    percent_5th = 0.30
    percent_6th = 0.20

    percent_math_5th = 0.60
    percent_science_5th = 1 - percent_math_5th

    percent_math_6th = 0.70
    percent_science_6th = 1 - percent_math_6th

    # 2. 계산 과정
    num_5th = total_students * percent_5th
    num_6th = total_students * percent_6th

    science_5th = num_5th * percent_science_5th
    science_6th = num_6th * percent_science_6th

    total_science = science_5th + science_6th

    # 3. 결과 반환
    return int(total_science)

# 결과 출력
print(solve_problem())

In [ ]:
# Ollama qwen2.5:3b 모델로 추론
pal_chain = pal_prompt | ollama
answer = pal_chain.invoke({"question": question})

print(answer.content)

In [ ]:
def solve_problem():
    # 변수 정의
    total_students = 500
    fifth_grade_percentage = 30 / 100
    sixth_grade_percentage = 20 / 100

    # 계산 과정
    fifth_graders = total_students * fifth_grade_percentage
    sixth_graders = total_students * sixth_grade_percentage
    
    fifth_grade_math_club = fifth_graders * (60 / 100)
    fifth_grade_science_club = fifth_graders - fifth_grade_math_club

    sixth_grade_math_club = sixth_graders * (70 / 100)
    sixth_grade_science_club = sixth_graders - sixth_grade_math_club
    
    # 결과 반환
    total_science_club_students = fifth_grade_science_club + sixth_grade_science_club

    return int(total_science_club_students)

# 문제 해결 함수 호출 및 결과 출력
science_club_students_count = solve_problem()
print(f"과학 동아리에는 {science_club_students_count}명의 학생이 있습니다.")

# **Reflexion**

* Reflexion은 AI가 자신의 이전 답변을 스스로 검토하고 평가하여 개선하는 메타인지적 프롬프팅 기법으로, 이를 통해 응답의 질을 점진적으로 향상시킬 수 있습니다.

* 이 방법은 AI가 자신의 답변에서 부족한 점, 오류, 또는 개선이 필요한 부분을 스스로 찾아내고 수정하도록 함으로써, 더 정확하고 완성도 높은 답변을 도출할 수 있게 합니다. 특히 복잡한 분석이나 창의적인 작업에서 효과적입니다.

* Reflexion의 강점은 AI가 자기 평가를 통해 지속적으로 개선된 결과물을 제공할 수 있다는 것이지만, 여러 번의 반복적인 검토와 수정 과정이 필요하므로 시간과 컴퓨팅 자원이 더 많이 소요될 수 있다는 제한점이 있습니다.

* 이 기법은 특히 글쓰기, 코드 리뷰, 분석 리포트 작성 등 높은 품질의 출력이 요구되는 작업에서 매우 유용하게 활용될 수 있습니다.

- 논문: https://arxiv.org/abs/2303.11366

In [ ]:
from langchain_core.prompts import PromptTemplate

# 프롬프트 템플릿 생성
reflexion_template = """
다음 문제에 대해 단계적으로 해결하여 초기 답안을 작성하고, 자체 평가 후 개선하시오:

문제: {question}

1단계: 초기 답안
---
[여기에 첫 번째 답안 작성]

2단계: 자체 평가
---
- 정확성 검토
- 논리적 오류 확인
- 설명의 명확성 평가
- 개선이 필요한 부분 식별

3단계: 개선된 답안
---
[평가를 바탕으로 개선된 답안 작성]

답안:
"""

reflexion_prompt = PromptTemplate(
    input_variables=["question"],
    template=reflexion_template
)

# 테스트용 문제
question = """
학교에서 500명의 학생이 있습니다. 이 중 30%는 5학년이고, 20%는 6학년 학생입니다. 
5학년 학생들 중 60%는 수학 동아리에 있고, 나머지는 과학 동아리에 있습니다. 
6학년 학생들 중 70%는 수학 동아리에 있고, 나머지는 과학 동아리에 있습니다.
과학 동아리에는 몇 명의 학생이 있나요?
"""

# OpenAI gpt-4.1-nano 모델로 답안 생성
reflexion_chain = reflexion_prompt | llm
answer = reflexion_chain.invoke({"question": question})

print(answer.content)

In [ ]:
# Ollama qwen2.5:3b 모델로 추론
reflexion_chain = reflexion_prompt | ollama
answer = reflexion_chain.invoke({"question": question})

print(answer.content)